In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import geopandas as gp

In [2]:
# Set file path
csvpath = "Resources/2016-precinct-president.csv"

# Read csv and preview dataframe
prec_df = pd.read_csv(csvpath, encoding="ISO-8859-1")
pd.set_option('display.max_columns', None)
prec_df.head()

C:\Users\test\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,17,19,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,year,stage,special,state,state_postal,state_fips,state_icpsr,county_name,county_fips,county_ansi,county_lat,county_long,jurisdiction,precinct,candidate,candidate_normalized,office,district,writein,party,mode,votes,candidate_opensecrets,candidate_wikidata,candidate_party,candidate_last,candidate_first,candidate_middle,candidate_full,candidate_suffix,candidate_nickname,candidate_fec,candidate_fec_name,candidate_google,candidate_govtrack,candidate_icpsr,candidate_maplight
0,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,32.532237,-86.64644,Autauga,10 JONES COMMUNITY CTR,Hillary Clinton,clinton,US President,statewide,False,democratic,election day,135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P00003392,"CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE",NaN,NaN,NaN,NaN
1,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,32.532237,-86.64644,Autauga,10 JONES COMMUNITY CTR,Gary Johnson,johnson,US President,statewide,False,independent,election day,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P60012234,"JOHNSON, JOHN FITZGERALD MR.",NaN,NaN,NaN,NaN
2,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,32.532237,-86.64644,Autauga,10 JONES COMMUNITY CTR,Jill Stein,stein,US President,statewide,False,independent,election day,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P20003984,"STEIN, JILL",NaN,NaN,NaN,NaN
3,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,32.532237,-86.64644,Autauga,10 JONES COMMUNITY CTR,Donald Trump,trump,US President,statewide,False,republican,election day,218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P80001571,"TRUMP, DONALD J. / MICHAEL R. PENCE",NaN,NaN,NaN,NaN
4,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,32.532237,-86.64644,Autauga,10 JONES COMMUNITY CTR,[Write-in],in,US President,statewide,True,NaN,election day,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Exploring the data

In [ ]:
# List of Columns
prec_df.columns

In [ ]:
# Getting the value counts for all the states
prec_df['state'].value_counts()

# Minnesota

In [3]:
# Only interested in Minnesota precincts
mn_df = prec_df.loc[prec_df['state'] == 'Minnesota']
mn_df.head()

,year,stage,special,state,state_postal,state_fips,state_icpsr,county_name,county_fips,county_ansi,county_lat,county_long,jurisdiction,precinct,candidate,candidate_normalized,office,district,writein,party,mode,votes,candidate_opensecrets,candidate_wikidata,candidate_party,candidate_last,candidate_first,candidate_middle,candidate_full,candidate_suffix,candidate_nickname,candidate_fec,candidate_fec_name,candidate_google,candidate_govtrack,candidate_icpsr,candidate_maplight
1124002,2016,gen,False,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,Darrell Castle,castle,US President,statewide,False,constitution,total,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P60021102,"CASTLE, DARRELL LANE",NaN,NaN,NaN,NaN
1124003,2016,gen,False,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,Hillary Clinton,clinton,US President,statewide,False,Democratic-Farmer-Labor,total,154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P00003392,"CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE",NaN,NaN,NaN,NaN
1124004,2016,gen,False,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,Rocky De La Fuente,fuente,US President,statewide,False,american delta,total,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P60016342,"DE LA FUENTE, ROQUE ROCKY",NaN,NaN,NaN,NaN
1124005,2016,gen,False,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,Gary Johnson,johnson,US President,statewide,False,libertarian,total,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P60012234,"JOHNSON, JOHN FITZGERALD MR.",NaN,NaN,NaN,NaN
1124006,2016,gen,False,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,Jill Stein,stein,US President,statewide,False,green,total,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P20003984,"STEIN, JILL",NaN,NaN,NaN,NaN


In [4]:
# Keep only the columns we're interested in
# Want to come back and remove more once I get a little deeper into it
mn_df = mn_df[['year', 'state', 'state_postal', 'state_fips', 'state_icpsr', 'county_name',
               'county_fips', 'county_ansi', 'county_lat', 'county_long', 'jurisdiction',
               'precinct', 'candidate_normalized', 'mode', 'votes', 'candidate_fec']]
mn_df.head(11)

,year,state,state_postal,state_fips,state_icpsr,county_name,county_fips,county_ansi,county_lat,county_long,jurisdiction,precinct,candidate_normalized,mode,votes,candidate_fec
1124002,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,castle,total,1,P60021102
1124003,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,clinton,total,154,P00003392
1124004,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,fuente,total,0,P60016342
1124005,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,johnson,total,18,P60012234
1124006,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,stein,total,5,P20003984
1124007,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,trump,total,356,P80001571
1124008,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,in,total,4,NaN
1124009,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,mcmullin,total,7,P60022654
1124010,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,kennedy,total,0,NaN
1124011,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,vacek,total,2,NaN


In [5]:
# Create a new column for Total Votes and add that to the dataframe

# Create the column and give it an initial value of zero
mn_df['total_votes'] = 0

# Add the new column into the dataframe
mn_df = mn_df[['year', 'state', 'state_postal', 'state_fips', 'state_icpsr', 'county_name',
               'county_fips', 'county_ansi', 'county_lat', 'county_long', 'jurisdiction',
               'precinct', 'candidate_normalized', 'mode', 'votes', 'total_votes', 'candidate_fec']]

# Preview
mn_df.head()

,year,state,state_postal,state_fips,state_icpsr,county_name,county_fips,county_ansi,county_lat,county_long,jurisdiction,precinct,candidate_normalized,mode,votes,total_votes,candidate_fec
1124002,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,castle,total,1,0,P60021102
1124003,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,clinton,total,154,0,P00003392
1124004,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,fuente,total,0,0,P60016342
1124005,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,johnson,total,18,0,P60012234
1124006,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,stein,total,5,0,P20003984


In [6]:
# Groupby the precincts
mn_groupby = mn_df.groupby(['precinct']).sum()
mn_groupby

,year,state_fips,state_icpsr,county_fips,county_ansi,county_lat,county_long,votes,total_votes
precinct,,,,,,,,,
0005 - 1-A BAUDETTE UNORG|5,20160,270,330,270770.0,6594830.0,487.68100,-949.04633,121,0
0010 - 1-B BAUDETTE CITY|10,20160,270,330,270770.0,6594830.0,487.68100,-949.04633,272,0
0015 - 2-A GUDRID UNORG|15,20160,270,330,270770.0,6594830.0,487.68100,-949.04633,124,0
0020 - 2-B BAUDETTE CITY|20,20160,270,330,270770.0,6594830.0,487.68100,-949.04633,229,0
0030 - 3-A UNORG|30,20160,270,330,270770.0,6594830.0,487.68100,-949.04633,199,0
...,...,...,...,...,...,...,...,...,...
ZUMBRO FALLS|170,20160,270,330,271570.0,6595230.0,442.89693,-922.33340,86,0
ZUMBRO TWP.|165,20160,270,330,271570.0,6595230.0,442.89693,-922.33340,446,0
ZUMBROTA P-1|210,20160,270,330,270490.0,6594700.0,444.06178,-927.16000,839,0


In [7]:
# Merge the two dataframes, mn_groupby and mn_df
merged_df = mn_df.merge(mn_groupby, on = "precinct")

# Preview
merged_df

,year_x,state,state_postal,state_fips_x,state_icpsr_x,county_name,county_fips_x,county_ansi_x,county_lat_x,county_long_x,jurisdiction,precinct,candidate_normalized,mode,votes_x,total_votes_x,candidate_fec,year_y,state_fips_y,state_icpsr_y,county_fips_y,county_ansi_y,county_lat_y,county_long_y,votes_y,total_votes_y
0,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.419760,Aitkin,AITKIN TWP|10,castle,total,1,0,P60021102,20160,270,330,270010.0,6631980.0,466.02446,-934.19760,547,0
1,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.419760,Aitkin,AITKIN TWP|10,clinton,total,154,0,P00003392,20160,270,330,270010.0,6631980.0,466.02446,-934.19760,547,0
2,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.419760,Aitkin,AITKIN TWP|10,fuente,total,0,0,P60016342,20160,270,330,270010.0,6631980.0,466.02446,-934.19760,547,0
3,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.419760,Aitkin,AITKIN TWP|10,johnson,total,18,0,P60012234,20160,270,330,270010.0,6631980.0,466.02446,-934.19760,547,0
4,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.419760,Aitkin,AITKIN TWP|10,stein,total,5,0,P20003984,20160,270,330,270010.0,6631980.0,466.02446,-934.19760,547,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41195,2016,Minnesota,MN,27,33,Yellow Medicine County,27173.0,659531.0,44.715736,-95.862756,Yellow Medicine,WOOD LAKE|160,trump,total,138,0,P80001571,20160,270,330,271730.0,6595310.0,447.15736,-958.62756,209,0
41196,2016,Minnesota,MN,27,33,Yellow Medicine County,27173.0,659531.0,44.715736,-95.862756,Yellow Medicine,WOOD LAKE|160,in,total,0,0,NaN,20160,270,330,271730.0,6595310.0,447.15736,-958.62756,209,0
41197,2016,Minnesota,MN,27,33,Yellow Medicine County,27173.0,659531.0,44.715736,-95.862756,Yellow Medicine,WOOD LAKE|160,mcmullin,total,6,0,P60022654,20160,270,330,271730.0,6595310.0,447.15736,-958.62756,209,0
41198,2016,Minnesota,MN,27,33,Yellow Medicine County,27173.0,659531.0,44.715736,-95.862756,Yellow Medicine,WOOD LAKE|160,kennedy,total,0,0,NaN,20160,270,330,271730.0,6595310.0,447.15736,-958.62756,209,0


In [8]:
# Pull out the desired columns
merged_df = merged_df[['year_x', 'state', 'state_postal', 'county_name', 'county_fips_x', 'county_ansi_x',
                       'county_lat_x', 'county_long_x', 'jurisdiction', 'precinct', 'candidate_normalized', 
                       'votes_x', 'votes_y']]
merged_df

,year_x,state,state_postal,county_name,county_fips_x,county_ansi_x,county_lat_x,county_long_x,jurisdiction,precinct,candidate_normalized,votes_x,votes_y
0,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.419760,Aitkin,AITKIN TWP|10,castle,1,547
1,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.419760,Aitkin,AITKIN TWP|10,clinton,154,547
2,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.419760,Aitkin,AITKIN TWP|10,fuente,0,547
3,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.419760,Aitkin,AITKIN TWP|10,johnson,18,547
4,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.419760,Aitkin,AITKIN TWP|10,stein,5,547
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41195,2016,Minnesota,MN,Yellow Medicine County,27173.0,659531.0,44.715736,-95.862756,Yellow Medicine,WOOD LAKE|160,trump,138,209
41196,2016,Minnesota,MN,Yellow Medicine County,27173.0,659531.0,44.715736,-95.862756,Yellow Medicine,WOOD LAKE|160,in,0,209
41197,2016,Minnesota,MN,Yellow Medicine County,27173.0,659531.0,44.715736,-95.862756,Yellow Medicine,WOOD LAKE|160,mcmullin,6,209
41198,2016,Minnesota,MN,Yellow Medicine County,27173.0,659531.0,44.715736,-95.862756,Yellow Medicine,WOOD LAKE|160,kennedy,0,209


In [9]:
# Rename the columns
merged_df = merged_df.rename(columns={'year_x': 'year', 'county_fips_x': 'county_fips',
                                      'county_ansi_x': 'county_ansi', 'county_lat_x': 'county_lat',
                                      'county_long_x': 'county_long', 'candidate_normalized': 'candidate',
                                      'votes_x': 'votes', 'votes_y': 'total_votes'})
merged_df

,year,state,state_postal,county_name,county_fips,county_ansi,county_lat,county_long,jurisdiction,precinct,candidate,votes,total_votes
0,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.419760,Aitkin,AITKIN TWP|10,castle,1,547
1,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.419760,Aitkin,AITKIN TWP|10,clinton,154,547
2,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.419760,Aitkin,AITKIN TWP|10,fuente,0,547
3,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.419760,Aitkin,AITKIN TWP|10,johnson,18,547
4,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.419760,Aitkin,AITKIN TWP|10,stein,5,547
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41195,2016,Minnesota,MN,Yellow Medicine County,27173.0,659531.0,44.715736,-95.862756,Yellow Medicine,WOOD LAKE|160,trump,138,209
41196,2016,Minnesota,MN,Yellow Medicine County,27173.0,659531.0,44.715736,-95.862756,Yellow Medicine,WOOD LAKE|160,in,0,209
41197,2016,Minnesota,MN,Yellow Medicine County,27173.0,659531.0,44.715736,-95.862756,Yellow Medicine,WOOD LAKE|160,mcmullin,6,209
41198,2016,Minnesota,MN,Yellow Medicine County,27173.0,659531.0,44.715736,-95.862756,Yellow Medicine,WOOD LAKE|160,kennedy,0,209


In [10]:
# Add a percent column which will determine how much percent of the precinct's vote each candidate got
merged_df['percent'] = (merged_df['votes'] / merged_df['total_votes']) * 100
merged_df.head(20)

,year,state,state_postal,county_name,county_fips,county_ansi,county_lat,county_long,jurisdiction,precinct,candidate,votes,total_votes,percent
0,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,castle,1,547,0.182815
1,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,clinton,154,547,28.153565
2,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,fuente,0,547,0.000000
3,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,johnson,18,547,3.290676
4,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,stein,5,547,0.914077
5,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,trump,356,547,65.082267
6,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,in,4,547,0.731261
7,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,mcmullin,7,547,1.279707
8,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,kennedy,0,547,0.000000
9,2016,Minnesota,MN,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,vacek,2,547,0.365631


# GeoPandas

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gp
from geopandas import GeoDataFrame

In [12]:
# load shp file extracted from zipfile
prec_shp = gp.read_file("Resources/bdry_votingdistricts.shp" )
prec_shp.head()

,VTDID,PCTNAME,PCTCODE,SHORTLABEL,MCDNAME,MCDCODE,MCDFIPS,MCDGNIS,CTU_TYPE,COUNTYNAME,COUNTYCODE,COUNTYFIPS,CONGDIST,MNSENDIST,MNLEGDIST,CTYCOMDIST,JUDDIST,SWCDIST,SWCDIST_N,WARD,HOSPDIST,HOSPDIST_N,PARKDIST,PARKDIST_N,geometry
0,271730045,Friendship Twp,0045,None,Friendship Twp,035,22832,00664234,township,Yellow Medicine,87,173,7,16,16A,3,08,5173,Yellow Medicine,None,None,None,None,None,"POLYGON ((284369.269 4964918.229, 284321.301 4..."
1,270910110,Galena Twp,0110,None,Galena Twp,055,23030,00664240,township,Martin,46,091,1,23,23A,5,05,6091,Martin,None,None,None,None,None,"POLYGON ((362977.491 4856419.851, 362974.359 4..."
2,270930045,Darwin Twp,0045,None,Darwin Twp,040,14860,00663929,township,Meeker,47,093,7,18,18A,1,08,2093,Meeker,None,None,None,None,None,"POLYGON ((391605.300 5000799.731, 391581.799 4..."
3,271370060,Biwabik Twp,0060,None,Biwabik Twp,060,06166,00663605,township,St. Louis,69,137,8,6,6B,4,06,3137,North St. Louis,None,None,None,None,None,"MULTIPOLYGON (((544403.320 5260705.800, 544214..."
4,270010015,Ball Bluff Twp,0015,None,Ball Bluff Twp,015,03358,00663498,township,Aitkin,1,001,8,10,10B,5,09,3001,Aitkin,None,None,None,None,None,"POLYGON ((485475.521 5207980.031, 485475.766 5..."


In [13]:
# Rename the columns that I'm going to keep
prec_shp = prec_shp.rename(columns = {'PCTNAME': 'precinct',
                                      'COUNTYFIPS': 'county_fips'})
prec_shp.head()

,VTDID,precinct,PCTCODE,SHORTLABEL,MCDNAME,MCDCODE,MCDFIPS,MCDGNIS,CTU_TYPE,COUNTYNAME,COUNTYCODE,county_fips,CONGDIST,MNSENDIST,MNLEGDIST,CTYCOMDIST,JUDDIST,SWCDIST,SWCDIST_N,WARD,HOSPDIST,HOSPDIST_N,PARKDIST,PARKDIST_N,geometry
0,271730045,Friendship Twp,0045,None,Friendship Twp,035,22832,00664234,township,Yellow Medicine,87,173,7,16,16A,3,08,5173,Yellow Medicine,None,None,None,None,None,"POLYGON ((284369.269 4964918.229, 284321.301 4..."
1,270910110,Galena Twp,0110,None,Galena Twp,055,23030,00664240,township,Martin,46,091,1,23,23A,5,05,6091,Martin,None,None,None,None,None,"POLYGON ((362977.491 4856419.851, 362974.359 4..."
2,270930045,Darwin Twp,0045,None,Darwin Twp,040,14860,00663929,township,Meeker,47,093,7,18,18A,1,08,2093,Meeker,None,None,None,None,None,"POLYGON ((391605.300 5000799.731, 391581.799 4..."
3,271370060,Biwabik Twp,0060,None,Biwabik Twp,060,06166,00663605,township,St. Louis,69,137,8,6,6B,4,06,3137,North St. Louis,None,None,None,None,None,"MULTIPOLYGON (((544403.320 5260705.800, 544214..."
4,270010015,Ball Bluff Twp,0015,None,Ball Bluff Twp,015,03358,00663498,township,Aitkin,1,001,8,10,10B,5,09,3001,Aitkin,None,None,None,None,None,"POLYGON ((485475.521 5207980.031, 485475.766 5..."


In [14]:
prec_shp = prec_shp[['precinct', 'county_fips', 'geometry']]
prec_shp

,precinct,county_fips,geometry
0,Friendship Twp,173,"POLYGON ((284369.269 4964918.229, 284321.301 4..."
1,Galena Twp,091,"POLYGON ((362977.491 4856419.851, 362974.359 4..."
2,Darwin Twp,093,"POLYGON ((391605.300 5000799.731, 391581.799 4..."
3,Biwabik Twp,137,"MULTIPOLYGON (((544403.320 5260705.800, 544214..."
4,Ball Bluff Twp,001,"POLYGON ((485475.521 5207980.031, 485475.766 5..."
...,...,...,...
4105,Borgholm Twp,095,"MULTIPOLYGON (((450879.660 5068471.679, 451005..."
4106,Corcoran P-2,053,"POLYGON ((455732.119 5000074.516, 457351.820 5..."
4107,Albertville P-1,171,"POLYGON ((447828.738 5007699.654, 447826.627 5..."
4108,Monticello P-2,171,"POLYGON ((439564.488 5015565.645, 439569.764 5..."


In [19]:
prec_shp['county_fips'] = prec_shp['county_fips'].astype(float)
prec_shp.dtypes

precinct         object
county_fips     float64
geometry       geometry
dtype: object

In [24]:
shp_merge = prec_shp.merge(merged_df, how='outer')

In [25]:
shp_merge

,precinct,county_fips,geometry,year,state,state_postal,county_name,county_ansi,county_lat,county_long,jurisdiction,candidate,votes,total_votes,percent
0,Friendship Twp,173.0,"POLYGON ((284369.269 4964918.229, 284321.301 4...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Galena Twp,91.0,"POLYGON ((362977.491 4856419.851, 362974.359 4...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Darwin Twp,93.0,"POLYGON ((391605.300 5000799.731, 391581.799 4...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Biwabik Twp,137.0,"MULTIPOLYGON (((544403.320 5260705.800, 544214...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ball Bluff Twp,1.0,"POLYGON ((485475.521 5207980.031, 485475.766 5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45305,WOOD LAKE|160,27173.0,None,2016.0,Minnesota,MN,Yellow Medicine County,659531.0,44.715736,-95.862756,Yellow Medicine,trump,138.0,209.0,66.028708
45306,WOOD LAKE|160,27173.0,None,2016.0,Minnesota,MN,Yellow Medicine County,659531.0,44.715736,-95.862756,Yellow Medicine,in,0.0,209.0,0.000000
45307,WOOD LAKE|160,27173.0,None,2016.0,Minnesota,MN,Yellow Medicine County,659531.0,44.715736,-95.862756,Yellow Medicine,mcmullin,6.0,209.0,2.870813
45308,WOOD LAKE|160,27173.0,None,2016.0,Minnesota,MN,Yellow Medicine County,659531.0,44.715736,-95.862756,Yellow Medicine,kennedy,0.0,209.0,0.000000
